<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [37]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
100% 84.9M/84.9M [00:00<00:00, 132MB/s] 
Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Импортируем нужные библиотеки:

In [38]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [39]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

# Обработка данных

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
data["flag"].unique()

array(['NO_QUERY'], dtype=object)

In [ ]:
data['emotion'].value_counts()

4    800000
0    800000
Name: emotion, dtype: int64

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
print(data.shape[0])

1600000


In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Нужно попробовать другую.   
p.s. другую так и не попробовал, потому что решил все протестировать сначала с одной, а для другой времени не выделил:(  
  
Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**.

In [ ]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text.lower())
        filtered_tokens = [t for t in tokens if all(c not in string.punctuation for c in t) and len(t) > 3]
        return filtered_tokens

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(t) - self.mean) / self.std for t in tokens if t in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]
# Почему sum???????
# Если делать mean, то получается другая форма на графике
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
type(examples['features'])

list

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [40]:
from sklearn.decomposition import PCA


from sklearn.manifold import TSNE
examples["transformed_features"] = TSNE(n_components=2).fit_transform(X=examples['features'])

# pca = PCA(n_components=2)
# examples["transformed_features"] = pca.fit_transform(X=examples['features'])  # Обучи PCA на эмбеддингах слов

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
# PCA
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

In [41]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1232', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Определим функции для тренировки и теста модели:

In [ ]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой ход
        outputs = model(features)

        # лосс
        # targets = targets.unsqueeze(1)
        # targets = targets.float()
        loss = criterion(outputs, targets) 
        # обратный ход
        loss.backward()
        # обновление весов
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            # считаем выход модели
            outputs = model(features)

            
            loss = criterion(outputs, targets) # Посчитай лосс
            # Получаем предсказания
            _, preds = torch.max(outputs, 1)
            acc = (preds == targets).to(torch.float).mean()  # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
# в случае BCELoss num_classes = 1
num_classes = 2
lr = 1e-2
num_epochs = 2
# изменение модели значимого качество не давало
model = nn.Sequential(
    nn.Linear(vector_size, 300),
    nn.ReLU(),
    
    nn.Linear(300, 400),
    nn.ReLU(),

    nn.Linear(400, 100),
    nn.ReLU(),

    nn.Linear(100, num_classes),
) # Твоя модель
model = model.cuda()
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

В данном случае можно использовать accuracy, т.к. классы сбалансированны
Но, возможно я что-то не понимаю, мы когда используем другую метрику, мы так или иначе улучшаем точность. Да, там у каждой свои преимущества, но все же.
Когда писал это, понял, что у нас всего два класса и можно использовать бинарную кросс энтропию в качестве лосса

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5193222169876098, 'Test Acc': 0.74221484375}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5109333018064499, 'Test Acc': 0.7465}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5120721242298334, 'Test Acc': 0.7454822284345048}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

# 1 Способ - сумма соседних эмбеддингов

Пробегаемся по всем токенам, если он не содержится в word2vec, то заменяем на <UNK>, и добавляем нулевой embedding, чтобы размер списка embeddings был одинаковый с размером списка tokens

In [ ]:
class TwitterDatasetContextEmbeddings(TwitterDataset):

    def get_embeddings_(self, tokens):

        embeddings = []
        for i, t in enumerate(tokens):

            # Если токен уже есть, то просто нормализуем и добавляем в список эмбеддингов
            if t in self.word2vec:
                
                emb = (self.word2vec.get_vector(t) - self.mean) / self.std
                embeddings.append(emb)

            # если же нет, то заменяем, вместо него нулевой вектор ставим
            else:
                tokens[i] = '<UNK>'
                embeddings.append([0 for _ in range(self.word2vec.vector_size)])

        # теперь заменяем на среднее соседей
        for i, t in enumerate(tokens):

            if t == '<UNK>':
                if i == 0:
                    try:
                        emb = embeddings[i+1]
                        # чтобы избежать случая только одного токена
                    except IndexError:
                        # если он один, то оставляем нулями
                        emb = [0 for _ in range(self.word2vec.vector_size)]
                elif i == len(tokens)-1:
                    # если последний, то приравниваем значение предыдущего
                    emb = embeddings[i-1]
                else:
                    # иначе считаем среднее
                    emb = np.mean([embeddings[i-1], embeddings[i+1]], axis=0)

                # делаем нормализацию, добавляем значение
                emb = (emb - self.mean) / self.std
                embeddings[i] = emb
            else:
                pass

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
dev = TwitterDatasetContextEmbeddings(dev_data, "text", "emotion", word2vec)

In [ ]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой ход
        outputs = model(features)

        # Получи предсказания модели
        loss = criterion(outputs, targets) # Посчитай лосс
        # Обнови параметры модели
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            outputs = model(features)

            # Получи предсказания модели
            loss = criterion(outputs, targets) # Посчитай лосс

            _, preds = torch.max(outputs, 1)
            acc = (preds == targets).to(torch.float).mean()  # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [ ]:
# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = nn.Sequential(
    nn.Linear(vector_size, 300),
    nn.ReLU(),
    
    nn.Linear(300, 400),
    nn.ReLU(),

    nn.Linear(400, 100),
    nn.ReLU(),

    nn.Linear(100, num_classes),
) # Твоя модель
model = model.cuda()
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Твой оптимайзер

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.512207615852356, 'Test Acc': 0.74671875}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5060415420532226, 'Test Acc': 0.74948046875}


In [ ]:
test_loader = DataLoader(
    TwitterDatasetContextEmbeddings(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 11535.61929712604, 'Test Acc': 0.7192429612619808}


# 2 способ - эмбеддинги из TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.extmath import randomized_svd

In [ ]:
def fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fun,
    preprocessor=fun,
    token_pattern=None)

In [ ]:
def my_get_tokens_(text, my_tokenizer=nltk.WordPunctTokenizer()):
    # Получи все токены из текста и профильтруй их
    tokens = my_tokenizer.tokenize(text.lower())
    filtered_tokens = [t for t in tokens if all(c not in string.punctuation for c in t) and len(t) > 3]
    return filtered_tokens

In [ ]:
examples = data["text"].sample(1000)

In [ ]:
examples

541200                @chrishasboobs AHHH I HOPE YOUR OK!!! 
750        @misstoriblack cool , i have no tweet apps  fo...
766711     @TiannaChaos i know  just family drama. its la...
285055     School email won't open  and I have geography ...
705995                                upper airways problem 
                                 ...                        
338333     @girrlonthewing Ha, well you'd be surprised at...
109574     Some dark clouds in #indiavotes #indiavotes09 ...
1349309    @wolfgnards awesome. Thanks for letting me kno...
671510     I left my heart @holdenbeach  Hoping to go bac...
385755                              I wish I had the Sims 3 
Name: text, Length: 1000, dtype: object

In [ ]:
docs = [my_get_tokens_(text) for text in examples]
docs[::100]

[['chrishasboobs', 'ahhh', 'hope', 'your'],
 ['powersurf',
  'case',
  'designed',
  'someone',
  'idiot',
  'stole',
  'prove',
  'they',
  'want',
  'play',
  'hardball',
  'game'],
 ['missing'],
 ['laydeedelish', 'definately', 'strong'],
 ['victoriastevens', 'yeah', 'started'],
 ['pollypocket3', 'hate'],
 ['finally',
  'that',
  'know',
  'gets',
  'gold',
  'logie',
  'disappointed',
  'wasn',
  'rove',
  'though'],
 ['just', 'stepped', 'chicharrone', 'crumbs'],
 ['congrats',
  'ashleytisdale',
  'mileycyrus',
  'youll',
  'last',
  'song',
  'eventually',
  'girl',
  'dont',
  'worry'],
 ['someone',
  'needs',
  'bring',
  'drugs',
  'wish',
  'have',
  'sore',
  'throat',
  'longer',
  'advil',
  'tylenol',
  'seem',
  'work',
  'anymore']]

In [ ]:
tfidf_matrix = tfidf.fit_transform(docs)
print(tfidf.vocabulary_)

tfidf_matrix.todense().shape

{'chrishasboobs': 530, 'ahhh': 94, 'hope': 1290, 'your': 3176, 'misstoriblack': 1765, 'cool': 614, 'have': 1225, 'tweet': 2893, 'apps': 164, 'razr': 2200, 'tiannachaos': 2798, 'know': 1494, 'just': 1447, 'family': 956, 'drama': 812, 'lame': 1517, 'next': 1864, 'time': 2806, 'hang': 1205, 'with': 3098, 'guys': 1188, 'like': 1569, 'sleepover': 2502, 'whatever': 3069, 'call': 424, 'school': 2373, 'email': 875, 'open': 1931, 'geography': 1111, 'stuff': 2641, 'there': 2766, 'revise': 2275, 'stupid': 2642, 'upper': 2952, 'airways': 102, 'problem': 2142, 'going': 1144, 'miss': 1752, 'pastor': 1993, 'sermon': 2412, 'faith': 952, 'lunch': 1633, 'should': 2456, 'come': 575, 'piginthepoke': 2043, 'feeling': 977, 'that': 2754, 'gahh': 1092, 'peyton': 2022, 'needs': 1849, 'live': 1586, 'this': 2781, 'horrible': 1294, 'mrstessyman': 1815, 'thank': 2748, 'glad': 1132, 'product': 2145, 'review': 2273, 'site': 2486, 'enjoy': 886, 'knitting': 1492, 'perezhilton': 2012, 'zach': 3192, 'makes': 1654, 'sitt

(1000, 3202)

In [ ]:
tfidf_matrix.todense()[0].shape

(1, 3202)

In [ ]:
V, Sigma, UT = randomized_svd(tfidf_matrix, 
                              n_components=300,
                              n_iter=5,
                              random_state=None)

In [ ]:
exampleword2tfidf = {}
for v in tfidf.vocabulary_:
    exampleword2tfidf[v] = UT[:, tfidf.vocabulary_[v]]

In [ ]:
exampleword2tfidf['make'].shape

(300,)

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
dev_examples = dev_data['text']
dev_docs = [my_get_tokens_(text) for text in dev_examples]

In [ ]:
# X = V Sigma U^T
UT = randomized_svd(tfidf.fit_transform(dev_docs),
                    n_components=200,
                    )[2]

In [ ]:
dev_word2tfidf = {}
for v in tfidf.vocabulary_:
    dev_word2tfidf[v] = UT[:, tfidf.vocabulary_[v]]

In [ ]:
import pickle

f = open("dev_word2tfidf.pkl","wb")
pickle.dump(dev_word2tfidf,f)
f.close()

In [ ]:
len(dev_word2tfidf['make'])

200

In [ ]:
test_examples = test_data['text']
test_docs = [my_get_tokens_(text) for text in test_examples]

In [ ]:
# X = V Sigma U^T

UT = randomized_svd(tfidf.transform(test_docs),
                    n_components=200,
                    )[2]

In [ ]:
test_word2tfidf = {}
for v in tfidf.vocabulary_:
    test_word2tfidf[v] = UT[:, tfidf.vocabulary_[v]]

In [ ]:
import pickle

f = open("test_word2tfidf.pkl","wb")
pickle.dump(test_word2tfidf,f)
f.close()

In [ ]:
len(test_word2tfidf['make'])

200

In [ ]:
# check: they should have different representation because the context and documents in test
# are different
any(test_word2tfidf['make'] == dev_word2tfidf['make'])

False

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text.lower())
        filtered_tokens = [t for t in tokens if all(c not in string.punctuation for c in t) and len(t) > 3]
        return filtered_tokens

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(t) - self.mean) / self.std for t in tokens if t in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
class TwitterDatasetTFIDFEmbeddings(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                 word2tfidf):
        super().__init__(data, feature_column, target_column, word2vec)
        self.word2tfidf = word2tfidf


    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их

        embeddings_pretrained = np.zeros((len(tokens), self.word2vec.vector_size))
        embeddings_tfidf = np.zeros((len(tokens), 200)) # 300 did not fit into RAM
        
        for i, t in enumerate(tokens):
            
            if t in self.word2tfidf:
                embeddings_tfidf[i] = self.word2tfidf.get(t)
            if t in self.word2vec:
                embeddings_pretrained[i] = (self.word2vec.get_vector(t) - self.mean) / self.std

        embeddings = np.hstack([embeddings_pretrained, embeddings_tfidf])

        assert embeddings.shape[1] == 500

        if len(embeddings) == 0:
            embeddings = np.zeros((1, 500))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
class TwitterDatasetTFIDFEmbeddings(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                 word2tfidf):
        super().__init__(data, feature_column, target_column, word2vec)
        self.word2tfidf = word2tfidf


    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их

        embeddings_pretrained = np.zeros((len(tokens), self.word2vec.vector_size))
        embeddings_tfidf = np.zeros((len(tokens), 200)) # 300 did not fit into RAM
        
        for i, t in enumerate(tokens):
            
            if t in self.word2tfidf:
                embeddings_tfidf[i] = self.word2tfidf.get(t)
            if t in self.word2vec:
                embeddings_pretrained[i] = (self.word2vec.get_vector(t) - self.mean) / self.std

        embeddings = np.hstack([embeddings_pretrained, embeddings_tfidf])

        assert embeddings.shape[1] == 500

        if len(embeddings) == 0:
            embeddings = np.zeros((1, 500))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
import pickle

with open('dev_word2tfidf.pkl', 'rb') as f:
    word2tfidf = pickle.load(f)

In [ ]:
word2vec = api.load("word2vec-google-news-300")

In [ ]:
dev = TwitterDatasetTFIDFEmbeddings(
    data=dev_data,
    feature_column="text", target_column="emotion",
    word2vec=word2vec,
    word2tfidf=word2tfidf)

In [ ]:
len(dev)

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

# changed np.sum to np.mean
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}


print(len(examples["features"]), len(examples["features"][0]))

1280 500


In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import torch.nn as nn
from torch.optim import Adam


# TODO: try different parameters

# 300 from word2vec
# 200 from tfidf
vector_size = 500

num_classes = 2
lr = 1e-3
# после 3-х эпох переобучение начинается?
# построить графики
num_epochs = 3

# TODO: define the model, loss, and optimiser
model = nn.Sequential(
    nn.Linear(vector_size, 200),
    nn.ReLU(),

    nn.Linear(200, 100),
    nn.ReLU(),

    nn.Linear(100, num_classes),
)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой ход
        outputs = model(features)

        # Получи предсказания модели
        loss = criterion(outputs, targets) # Посчитай лосс
        # Обнови параметры модели
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            outputs = model(features)

            # Получи предсказания модели
            loss = criterion(outputs, targets) # Посчитай лосс

            _, preds = torch.max(outputs, 1)
            acc = (preds == targets).to(torch.float).mean()  # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.4990757074356079, 'Test Acc': 0.75308203125}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.4908881285190582, 'Test Acc': 0.75923046875}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.48445381903648377, 'Test Acc': 0.76330078125}


In [ ]:
import pickle
from torch.utils.data import DataLoader


# rewrite word2tfidf to save RAM
# with open('test_word2tfidf.pkl', 'rb') as f:
#     word2tfidf = pickle.load(f)

# create test loader
# note, we create the dataset inside
test_loader = DataLoader(
    TwitterDatasetTFIDFEmbeddings(test_data, "text", "emotion", word2vec,
                                  word2tfidf=word2tfidf), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.4854061859674728, 'Test Acc': 0.7633661142172524}


Вывод:

|Метод                     |   score     |
|---------------------------------|:-----------------|
|default                          |0.7454822284345048|   
|Модель с контекстным эмбеддингом |0.7115927016773163|  
|Модель с TFidf                   |0.763366114217252

Модель, в которой мы для неизвестных эмбеддингов работает хуже всех. Мне кажется, это происходит из-за случаев, когда рядом стоят несколько неизвестных токенов. Скорее всего, нужно было усреднять не по соседним, а по целому предложению, либо по окну.  
Модель с TFidf показала лучшее качество. В принципе, это логично - мы использовали для каждого слова его эмбеддинг из word2vec + информацию о вероятноисти встретить его в конкретно наших текстах.